## Sagemaker Training

In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import boto3
import os

In [2]:
s3_input_train  = sagemaker.TrainingInput(s3_data='s3://trends-marketplace-data/fraud-detection/train.csv', content_type='csv')
#s3_input_validation = sagemaker.s3_input(s3_data='s3://trendzmarket/segmaker/validation/validation.csv', content_type='csv')

In [3]:
from sagemaker.amazon.amazon_estimator import get_image_uri
sess = sagemaker.Session()
region = boto3.Session().region_name
container = get_image_uri(region, 'xgboost', repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [4]:
role = get_execution_role()

In [5]:
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path= 's3://trends-marketplace-data/fraud-detection/modelDir/model.joblib/',
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=10,\
                        objective='binary:logistic', \
                        eta =0.2,\
                        gamma=0,\
                        min_child_weight=1,\
                        max_delta_step=0,\
                        subsample=0.85, \
                        colsample_bytree=0.7,\
                        colsample_bylevel=1,\
                        alpha=0,\
                        scale_pos_weight=1,\
                        num_round = 100,\
                        seed=42)

xgb.fit({'train': s3_input_train})

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-04-30 03:16:30 Starting - Starting the training job...ProfilerReport-1651288590: InProgress
...
2022-04-30 03:17:13 Starting - Preparing the instances for training.........
2022-04-30 03:18:54 Downloading - Downloading input data...
2022-04-30 03:19:16 Training - Downloading the training image......
2022-04-30 03:20:17 Training - Training image download completed. Training in progress.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[03:20:22] 555719x7 matrix with 3890033 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Single

### Model Deployment

In [11]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.t2.medium', endpoint_name = 'fifa')

-------!

### Endpoint Deletion

In [12]:
xgb_predictor.delete_endpoint()